Replicação - Equipe

- snapshot_20230727
- pegaram 100 conversas para montar as categorias
- resultou em 22 categorias
- 95% de confiança e 5% de erro
- Qualquer conversa que não estiver disponível e não for em inglês, descarta (N/A)

Importando o dataset em json

In [ ]:
import re

import pandas as pd
import json

discussion_json_path = './datasets/20230727_195954_discussion_sharings.json'
hn_sharings_json_path = './datasets/20230727_195816_hn_sharings.json'

with open(discussion_json_path, "r", encoding="utf-8") as f:
    discussion_data = json.load(f)

with open(hn_sharings_json_path, "r", encoding="utf-8") as f:
    hn_sharings_data = json.load(f)

discussion_sources = discussion_data.get("Sources", [])
hn_sources = hn_sharings_data.get("Sources", [])

sources = discussion_sources + hn_sources
rows = []

percorre o JSON das fontes e conversas do ChatGPT, filtra apenas os prompts feitos pelos usuários e transforma cada prompt em uma linha estruturada com metadados para análise em um DataFrame.


In [ ]:
for src in sources:
    chat_sharings = src.get("ChatgptSharing", []) or []
    for share in chat_sharings:
        conversations = share.get("Conversations", []) or []
        for idx, turn in enumerate(conversations):
            prompt = turn.get("Prompt")
            if prompt is None:
                continue

            rows.append({
                "source_type": src.get("Type"),
                "source_url": src.get("URL"),
                "source_title": src.get("Title"),
                "chat_url": share.get("URL"),
                "chat_title": share.get("Title"),
                "date_of_conversation": share.get("DateOfConversation"),
                "status": share.get("Status"),
                "prompt_index": idx,
                "prompt": prompt
            })

df = pd.DataFrame(rows)


Limpeza do texto

In [ ]:
if not df.empty:
    df["prompt"] = (
        df["prompt"]
        .astype(str)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )
    df = df[df["prompt"].str.len() > 0].reset_index(drop=True)

print(f"Total de prompts extraídos: {len(df)}")
df.head()

In [ ]:
"""Tratamentos"""


# Função para remover os textos que não estão em inglês.
def is_english(text):
    if not isinstance(text, str):
        return False
    return bool(re.match(r"^[\x00-\x7F]+$", text))


# Aplicando o filtro no dataframe.
df["is_english"] = df["prompt"].apply(is_english)

df_lang = df[df["is_english"]].reset_index(drop=True)

print(f"Após filtro de idioma (inglês): {len(df_lang)}")

# Realizar uma última verificação para garantir que os dados estejam filtrados de maneira adequada.
df_final = df_lang[
    df_lang["prompt"].notna() & (df_lang["prompt"].str.len() > 5)
].reset_index(drop=True)

print(f"Dataset final após todos os filtros: {len(df_final)}")

df_final.head()